# Table of Contents
 <p>

In [1]:
import pandas as pd
from openpyxl import load_workbook
import datetime

In [2]:
#解决字符编码问题
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [3]:
header = [
    "ticket_record_id", "begin_station_id", "end_station_id", "unit_price",
    "tk_num", "userid", "original_type", "pay_type", "target", "description",
    "trade_no", "tal_fee", "time_end", "voucher", "err_status",
    "pay_result_code", "notify_type", "finish_num", "notify_time",
    "refund_type", "result_code", "refund_fee", "refund_time", "error_code",
    "error_detail", "order_status", "ticketurl", "create_time", "ticket_no",
    "station_code", "product_code"
]
data = pd.read_csv('./数据（1-6月）/APPdata/u_user_ticket_record_his.csv')
data.head()

,ticket_record_id,begin_station_id,end_station_id,unit_price,tk_num,userid,original_type,pay_type,target,description,...,refund_fee,refund_time,error_code,error_detail,order_status,ticketurl,create_time,ticket_no,station_code,product_code
0,0000086a-025f-11e7-bc41-005056900b11,408.0,314.0,500,1,NaN,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0408&site_end=0314&ticket_num=1&tic...,2017/3/6 19:12:52,78521027861444077184,NaN,NaN
1,00001488-2d89-11e7-bc41-005056900b11,825.0,410.0,300,1,e7e9f4d8-e6a9-4b65-9e7b-b7a088652239,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0825&site_end=0410&ticket_num=1&tic...,2017/4/30 17:34:35,12343063643511062005,NaN,NaN
2,000015dd-f1c6-11e6-ab56-005056900b11,314.0,622.0,200,1,NaN,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0314&site_end=0622&ticket_num=1&tic...,2017/2/13 16:30:47,59853988406836477639,NaN,NaN
3,000017f4-1f57-11e7-bc41-005056900b11,105.0,245.0,400,1,40d9f40e-65a5-42f6-8b75-46782c3143f2,3,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0105&site_end=0245&ticket_num=1&tic...,2017/4/12 16:06:39,12343282726191495265,NaN,NaN
4,00002d20-0403-11e7-bc41-005056900b11,302.0,3004.0,600,1,NaN,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0302&site_end=3004&ticket_num=1&tic...,2017/3/8 21:26:48,46721183805617024225,NaN,NaN


In [4]:
station = pd.read_csv('./数据（1-6月）/APPdata/ticket_station.csv')
station.head()

,id,stationid,cn_name,en_name
0,1,101,西朗,Xilang
1,2,102,坑口,Kengkou
2,3,103,花地湾,Huadiwan
3,4,104,芳村,Fangcun
4,5,105,黄沙,Huangsha


In [5]:
#数据格式处理
station['stationid'] = station['stationid'].apply(float)
data['time_end'] = pd.to_datetime(data['time_end'], format='%Y%m%d%H%M%S')
# print (data['time_end'])
data = data[data['result_code'] == 'SUCCESS']
data = data[data['order_status'] == 2]
station = station.drop_duplicates('stationid')

In [6]:
#添加开始／结束站点
# def add_station(a):
#     for j in range(len(station)):
#         if (a==station['stationid'][j]):
#             return station['cn_name'][j]

# data['begin_station']=data['begin_station_id'].apply(add_station)
# data['end_station']=data['end_station_id'].apply(add_station)

data = data.merge(
    station, left_on=['begin_station_id'], right_on=['stationid'], how='left')
data = data.rename(columns={'cn_name': 'begin_station'})
data = data.merge(
    station, left_on=['end_station_id'], right_on=['stationid'], how='left')
data = data.rename(columns={'cn_name': 'end_station'})

In [7]:
#excel写入函数
def excelAddSheet(dataframe, excelWriter, sheetname):
    book = load_workbook(excelWriter.path)
    excelWriter.book = book
    dataframe.to_excel(
        excel_writer=excelWriter, sheet_name=sheetname, index=None)
    excelWriter.close()


excelPath = './数据（1-6月）/APPdata/hot_station.xlsx'
excelWriter = pd.ExcelWriter(excelPath, engine='openpyxl')

In [8]:
#筛选热门起终点站
# print (data['begin_station_id'].value_counts())
# print (data['begin_station_id'].value_counts().mean())
hot_begin_station = data['begin_station'].value_counts()[
    data['begin_station'].value_counts() >
    data['begin_station_id'].value_counts().mean()]
# print (hot_begin_station.index[:10])
hot_begin_station = pd.DataFrame(
    hot_begin_station, index=hot_begin_station.index)
hot_begin_station['num'] = range(len(hot_begin_station))
hot_begin_station['hot_begin_station'] = hot_begin_station.index
hot_begin_station.index = hot_begin_station['num']
del hot_begin_station['num']
hot_begin_station.columns = [u'购票数量', u'站点']
# print (hot_begin_station)
excelAddSheet(hot_begin_station, excelWriter, 'hot_begin_station')

# print (data['end_station_id'].value_counts())
# print (data['end_station_id'].value_counts().mean())
hot_end_station = data[
    'end_station'].value_counts()[data['end_station'].value_counts() >
                                  data['end_station_id'].value_counts().mean()]
# print (hot_end_station.index[:10])
hot_end_station = pd.DataFrame(hot_end_station, index=hot_end_station.index)
hot_end_station['num'] = range(len(hot_end_station))
hot_end_station['hot_begin_station'] = hot_end_station.index
hot_end_station.index = hot_end_station['num']
del hot_end_station['num']
hot_end_station.columns = [u'购票数量', u'站点']
# print (hot_end_station)
excelAddSheet(hot_end_station, excelWriter, 'hot_end_station')

In [9]:
#热门起终点站客流走势分析
data['week_day'] = data['time_end'].apply(lambda x: x.dayofweek)
data['time_end_day'] = data['time_end'].apply(pd.Timestamp.date)

#广州南站
# print (data['time_end_day'][data['begin_station_id'] == 249.0].value_counts().sort_index())
# print (data['time_end_day'][data['end_station_id'] == 249.0].value_counts().sort_index())
print(data['time_end_day'][(data['begin_station_id'] == 249.0) & (
    (data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 249.0) &
#                             ((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().sort_index())
print(data['time_end_day'][(data['begin_station_id'] == 249.0) & (
    data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 114.0) &
#                             (data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().sort_index())

#体育西路
# print (data['time_end_day'][data['begin_station_id'] == 114.0].value_counts().sort_index())
# print (data['time_end_day'][data['end_station_id'] == 114.0].value_counts().sort_index())
print(data['time_end_day'][(data['begin_station_id'] == 114.0) & (
    (data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 114.0) &
#                             ((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().sort_index())
print(data['time_end_day'][(data['begin_station_id'] == 114.0) & (
    data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 114.0) &
#                             (data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().sort_index())

# print(hot_begin_station[u'站点ID'])

#热门起点站
with pd.ExcelWriter(
        './数据（1-6月）/APPdata/hot_begin_station_people.xlsx') as writer:
    for station in hot_begin_station[u'站点']:
        temp = data['time_end_day'][data['begin_station'] ==
                                    station].value_counts().sort_index()
        temp.name = u'购票量'
        temp.to_excel(writer, sheet_name=str(station))

#热门终点站
with pd.ExcelWriter(
        './数据（1-6月）/APPdata/hot_end_station_people.xlsx') as writer:
    for station in hot_end_station[u'站点']:
        temp = data['time_end_day'][data['end_station'] ==
                                    station].value_counts().sort_index()
        temp.name = u'购票量'
        temp.to_excel(writer, sheet_name=str(station))

#周末时期
with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_station_weekend.xlsx') as writer:
    for station in hot_end_station[u'站点']:
        temp = data['time_end_day'][
            (data['begin_station'] == station) & (data['week_day'] == 0) | (
                data['week_day'] == 6)].value_counts().sort_index()
        temp.name = u'购票量'
        temp.to_excel(writer, sheet_name=str(station))

In [10]:
#热门站点进站高峰期分析
data['time_end_hour'] = data['time_end'].apply(lambda x: x.hour)

#广州南站
print(data['time_end_hour'][data['begin_station_id'] == 249.0].value_counts()
      .sort_index())

#体育西路
print(data['time_end_hour'][data['begin_station_id'] == 114.0].value_counts()
      .sort_index())

#广州塔
print(data['time_end_hour'][data['begin_station_id'] == 305.0].value_counts()
      .sort_index())

# with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_station_hour.xlsx') as writer:
#     for station in hot_begin_station[u'站点']:
#         temp=data['time_end_hour'][data['begin_station'] == station].value_counts().sort_index()
#         temp.name=u'购票量'
#         temp.to_excel(writer, sheet_name=str(station))

with pd.ExcelWriter('./数据（1-6月）/APPdata/test.xlsx') as writer:
    temp1 = data['time_end_hour'][(data['begin_station_id'] == 305.0)
                                  & (data['time_end_day'] == datetime.date(
                                      2017, 5,
                                      1))].value_counts().sort_index()
    temp2 = data['time_end_hour'][(data['begin_station_id'] == 305.0)
                                  & (data['time_end_day'] == datetime.date(
                                      2017, 3,
                                      15))].value_counts().sort_index()
    temp = pd.concat([temp1, temp2])
    temp.to_excel(writer, sheet_name=str(249))

# with pd.ExcelWriter('./数据（1-6月）/APPdata/test.xlsx') as writer:
#     temp1=data['time_end_hour'][((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().sort_index()
#     temp2=data['time_end_hour'][((data['week_day'] != 0) & (data['week_day'] != 6))].value_counts().sort_index()
#     temp=pd.concat([temp1,temp2])
#     temp.to_excel(writer, sheet_name=str(249))

In [11]:
#热门线路统计
#按日期
hot_route = data.loc[:, [
    'time_end_day', 'begin_station', 'end_station', 'tk_num'
]]
hot_route = hot_route.groupby(
    ['time_end_day', 'begin_station', 'end_station'], as_index=False).sum()
# print (hot_route.loc[:,['begin_station','end_station','tk_num']][hot_route['time_end_day'] == pd.Timestamp('2017-05-01').date()].sort_values(by=['tk_num'],ascending=False))

# with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_route.xlsx') as writer:
#     for day in hot_route['time_end_day'].unique():
#         temp = hot_route.loc[:,['begin_station','end_station','tk_num']][hot_route['time_end_day'] == day].sort_values(by=['tk_num'],ascending=False)
#         temp.to_excel(writer, sheet_name=str(day))

#总量
hot_route = data.loc[:, ['begin_station', 'end_station', 'tk_num']]
hot_route = hot_route.groupby(['begin_station', 'end_station']).sum()
# print (hot_route)

In [12]:
#处理天气数据
weather = pd.read_csv('./数据（1-6月）/weather/weather.csv')
weather['date'] = pd.to_datetime(weather['date'], format='%Y-%m-%d')
weather = weather[weather.date > datetime.date(2017, 1, 17)]
weather.shape

(162, 6)

In [13]:
weather['hmean'] = (weather.hmax + weather.hmin) / 2
weather.hmean[weather.hmean >= 30].count()  # 26度及以上 56
weather.hmean[(weather.hmean >= 20)
              & (weather.hmean < 30)].count()  # 19度及以上26度以下 54
weather.hmean[weather.hmean < 20].count()  # 19度以下 52

61

In [15]:
weather['date'] = weather['date'].apply(pd.Timestamp.date)
data = data.merge(
    weather, left_on=['time_end_day'], right_on=['date'], how='left')
data.head()

,ticket_record_id,begin_station_id,end_station_id,unit_price,tk_num,userid,original_type,pay_type,target,description,...,week_day,time_end_day,time_end_hour,date,hmax,hmin,weather,wind_direction,wind_strength,hmean
0,0000086a-025f-11e7-bc41-005056900b11,408.0,314.0,500,1,NaN,4,NaN,TVM,NaN,...,0,2017-03-06,19,2017-03-06,18.0,14.0,小雨,东北风,2级,16.0
1,00001488-2d89-11e7-bc41-005056900b11,825.0,410.0,300,1,e7e9f4d8-e6a9-4b65-9e7b-b7a088652239,4,NaN,TVM,NaN,...,6,2017-04-30,17,2017-04-30,29.0,21.0,多云,东南风,3级,25.0
2,000015dd-f1c6-11e6-ab56-005056900b11,314.0,622.0,200,1,NaN,4,NaN,TVM,NaN,...,0,2017-02-13,16,2017-02-13,21.0,9.0,晴,北风,1级,15.0
3,000017f4-1f57-11e7-bc41-005056900b11,105.0,245.0,400,1,40d9f40e-65a5-42f6-8b75-46782c3143f2,3,NaN,TVM,NaN,...,2,2017-04-12,16,2017-04-12,20.0,17.0,中雨,北风,2级,18.5
4,00002d20-0403-11e7-bc41-005056900b11,302.0,3004.0,600,1,NaN,4,NaN,TVM,NaN,...,2,2017-03-08,21,2017-03-08,16.0,13.0,小雨,北风,1级,14.5


In [42]:
with pd.ExcelWriter('./数据（1-6月）/weather/tem_people.xlsx') as writer:
    temp1 = ((data['time_end_hour'][data['hmean'] >= 30].value_counts()) /
             (data['time_end_day'][data['hmean'] >= 30].duplicated()
              ).value_counts()[False]).sort_index()
    temp1 = temp1.apply(int)
    temp1.name = u'30度及以上'

    temp2 = ((data['time_end_hour'][(data['hmean'] < 30)
                                    & (data['hmean'] >= 20)].value_counts()) /
             (data['time_end_day'][(data['hmean'] < 30)
                                   & (data['hmean'] >= 20)].duplicated()
              ).value_counts()[False]).sort_index()
    temp2 = temp2.apply(int)
    temp2.name = u'20度及以上30度以下'

    temp3 = ((data['time_end_hour'][data['hmean'] < 20].value_counts()) /
             (data['time_end_day'][data['hmean'] < 20].duplicated()
              ).value_counts()[False]).sort_index()
    temp3 = temp3.apply(int)
    temp3.name = u'20度以下'

    temp = pd.concat([temp1, temp2, temp3], axis=1)
    temp.to_excel(writer, sheet_name=str('温度'))

In [26]:
def rain(x):
    if u'小雨' in str(x):
        return 1
    elif u'中雨' in str(x):
        return 1
    elif u'大雨' in str(x):
        return 2
    elif u'暴雨' in str(x):
        return 2
    else:
        return 0

In [27]:
weather['rain'] = weather['weather'].apply(rain)
data['rain'] = data['weather'].apply(rain)
data.head()

,ticket_record_id,begin_station_id,end_station_id,unit_price,tk_num,userid,original_type,pay_type,target,description,...,time_end_day,time_end_hour,date,hmax,hmin,weather,wind_direction,wind_strength,hmean,rain
0,0000086a-025f-11e7-bc41-005056900b11,408.0,314.0,500,1,NaN,4,NaN,TVM,NaN,...,2017-03-06,19,2017-03-06,18.0,14.0,小雨,东北风,2级,16.0,1
1,00001488-2d89-11e7-bc41-005056900b11,825.0,410.0,300,1,e7e9f4d8-e6a9-4b65-9e7b-b7a088652239,4,NaN,TVM,NaN,...,2017-04-30,17,2017-04-30,29.0,21.0,多云,东南风,3级,25.0,0
2,000015dd-f1c6-11e6-ab56-005056900b11,314.0,622.0,200,1,NaN,4,NaN,TVM,NaN,...,2017-02-13,16,2017-02-13,21.0,9.0,晴,北风,1级,15.0,0
3,000017f4-1f57-11e7-bc41-005056900b11,105.0,245.0,400,1,40d9f40e-65a5-42f6-8b75-46782c3143f2,3,NaN,TVM,NaN,...,2017-04-12,16,2017-04-12,20.0,17.0,中雨,北风,2级,18.5,1
4,00002d20-0403-11e7-bc41-005056900b11,302.0,3004.0,600,1,NaN,4,NaN,TVM,NaN,...,2017-03-08,21,2017-03-08,16.0,13.0,小雨,北风,1级,14.5,1


In [29]:
#test
# data['time_end_day'][1]
# 162 / 3
# weather.head()
# data.time_end_day[0] == weather.date[427]
# u'雨' in str(weather.weather[420])
# data[data['rain'] == 0].groupby(['time_end_day']).count()
# data[data['rain'] == 0].shape
# test = data['time_end_day'][data['rain'] == 1].duplicated()
# test = test.value_counts()[False]
# 'a' in ['ab', 'b']
# data['time_end_day'][data.hmean >= 30].duplicated().value_counts()[False]
# data.hmean[0]
# data.shape
# weather.weather[382]
# len(good_date)
# len(bad_date)
# bad_date
# data[data.time_end_day == datetime.date(2017, 6, 30)]
# data.loc[:, ['time_end_day', 'weather', 'hmax']][(data.time_end_day >= datetime.date(2017, 5, 1)) & (data.time_end_day <= datetime.date(2017, 6, 30))].groupby('time_end_day').sum()
# u'暴雨' in u'大到暴雨'
# data
# (data['time_end_day'][data['hmean'] >= 30].duplicated()).value_counts()[False]
data

,ticket_record_id,begin_station_id,end_station_id,unit_price,tk_num,userid,original_type,pay_type,target,description,...,time_end_day,time_end_hour,date,hmax,hmin,weather,wind_direction,wind_strength,hmean,rain
0,0000086a-025f-11e7-bc41-005056900b11,408.0,314.0,500,1,NaN,4,NaN,TVM,NaN,...,2017-03-06,19,2017-03-06,18.0,14.0,小雨,东北风,2级,16.0,1
1,00001488-2d89-11e7-bc41-005056900b11,825.0,410.0,300,1,e7e9f4d8-e6a9-4b65-9e7b-b7a088652239,4,NaN,TVM,NaN,...,2017-04-30,17,2017-04-30,29.0,21.0,多云,东南风,3级,25.0,0
2,000015dd-f1c6-11e6-ab56-005056900b11,314.0,622.0,200,1,NaN,4,NaN,TVM,NaN,...,2017-02-13,16,2017-02-13,21.0,9.0,晴,北风,1级,15.0,0
3,000017f4-1f57-11e7-bc41-005056900b11,105.0,245.0,400,1,40d9f40e-65a5-42f6-8b75-46782c3143f2,3,NaN,TVM,NaN,...,2017-04-12,16,2017-04-12,20.0,17.0,中雨,北风,2级,18.5,1
4,00002d20-0403-11e7-bc41-005056900b11,302.0,3004.0,600,1,NaN,4,NaN,TVM,NaN,...,2017-03-08,21,2017-03-08,16.0,13.0,小雨,北风,1级,14.5,1
5,00004caa-5015-11e7-819d-005056900b11,114.0,314.0,300,1,cf09de5e-7d25-4fca-8ea8-ad23f3c0aec8,4,NaN,TVM,NaN,...,2017-06-13,16,2017-06-13,32.0,26.0,中雨,东风,3级,29.0,1
6,00005707-2e08-11e7-bc41-005056900b11,6014.0,506.0,400,1,3b477579-5e0b-4543-8e8d-a350c8c3c6d2,4,NaN,TVM,NaN,...,2017-05-01,8,2017-05-01,30.0,23.0,多云,东南风,3级,26.5,0
7,00005b5e-60a1-11e7-819d-005056900b11,305.0,310.0,400,4,938b324d-b8d3-4d0c-91a8-1edc18131f9e,4,NaN,TVM,NaN,...,2017-07-04,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,00007b6e-181c-11e7-bc41-005056900b11,405.0,411.0,400,1,8b31c5d2-9466-4be6-80e7-48602bbff870,4,NaN,TVM,NaN,...,2017-04-03,11,2017-04-03,26.0,16.0,晴,南风,3级,21.0,0
9,00007ef6-0488-11e7-bc41-005056900b11,211.0,249.0,500,1,NaN,4,NaN,TVM,NaN,...,2017-03-09,13,2017-03-09,18.0,15.0,小雨,西南风,微风,16.5,1


In [31]:
with pd.ExcelWriter('./数据（1-6月）/weather/rain_people.xlsx') as writer:
    temp1 = ((data['time_end_hour'][data['rain'] == 0].value_counts()) /
             (data['time_end_day'][data['rain'] == 0].duplicated()
              ).value_counts()[False]).sort_index()
    temp1 = temp1.apply(int)
    temp1.name = u'无雨'

    temp2 = ((data['time_end_hour'][data['rain'] == 1].value_counts()) /
             (data['time_end_day'][data['rain'] == 1].duplicated()
              ).value_counts()[False]).sort_index()
    temp2 = temp2.apply(int)
    temp2.name = u'小到中雨'

    temp3 = ((data['time_end_hour'][data['rain'] == 2].value_counts()) /
             (data['time_end_day'][data['rain'] == 2].duplicated()
              ).value_counts()[False]).sort_index()
    temp3 = temp3.apply(int)
    temp3.name = u'大到暴雨'

    temp = pd.concat([temp1, temp2, temp3], axis=1)
    temp.to_excel(writer, sheet_name=str('雨量'))

In [32]:
def weather_good(x):
    good = [u'晴', u'阴', u'多云']
    bad = [u'大雨', u'暴雨']
    for i in range(len(good)):
        if good[i] in str(x):
            return 1
    for j in range(len(bad)):
        if bad[j] in str(x):
            return 0

In [33]:
good_date = []
bad_date = []

good = [u'晴', u'阴', u'多云']
bad = [u'大雨', u'暴雨']
for x in range(len(weather)):
    for i in range(len(good)):
        if good[i] in str(weather.weather[382 + x]):
            good_date.append(weather.date[382 + x])
    for j in range(len(bad)):
        if bad[j] in str(weather.weather[382 + x]):
            bad_date.append(weather.date[382 + x])
    if weather.hmean[382 + x] >= 30:
        bad_date.append(weather.date[382 + x])

good_date = list(set(good_date))
bad_date = list(set(bad_date))

In [34]:
def goodBad_date(x):
    if x in good_date:
        return 1
    if x in bad_date:
        return 0

In [35]:
def hot_weather(x):
    if x >= 35:
        return 1
    else:
        return 0


def heavyrain_weather(x):
    if u'暴雨' in str(x):
        return 1
    else:
        return 0


def typhoon_weather(x):
    typ_date = [
        datetime.date(2017, 6, 12),
        datetime.date(2017, 6, 13),
        datetime.date(2017, 6, 14)
    ]
    if x in typ_date:
        return 1
    else:
        return 0

In [36]:
# data['good'] = data['weather'].apply(weather_good)
data['good'] = data['time_end_day'].apply(goodBad_date)
data.head()

,ticket_record_id,begin_station_id,end_station_id,unit_price,tk_num,userid,original_type,pay_type,target,description,...,time_end_hour,date,hmax,hmin,weather,wind_direction,wind_strength,hmean,rain,good
0,0000086a-025f-11e7-bc41-005056900b11,408.0,314.0,500,1,NaN,4,NaN,TVM,NaN,...,19,2017-03-06,18.0,14.0,小雨,东北风,2级,16.0,1,NaN
1,00001488-2d89-11e7-bc41-005056900b11,825.0,410.0,300,1,e7e9f4d8-e6a9-4b65-9e7b-b7a088652239,4,NaN,TVM,NaN,...,17,2017-04-30,29.0,21.0,多云,东南风,3级,25.0,0,1.0
2,000015dd-f1c6-11e6-ab56-005056900b11,314.0,622.0,200,1,NaN,4,NaN,TVM,NaN,...,16,2017-02-13,21.0,9.0,晴,北风,1级,15.0,0,1.0
3,000017f4-1f57-11e7-bc41-005056900b11,105.0,245.0,400,1,40d9f40e-65a5-42f6-8b75-46782c3143f2,3,NaN,TVM,NaN,...,16,2017-04-12,20.0,17.0,中雨,北风,2级,18.5,1,NaN
4,00002d20-0403-11e7-bc41-005056900b11,302.0,3004.0,600,1,NaN,4,NaN,TVM,NaN,...,21,2017-03-08,16.0,13.0,小雨,北风,1级,14.5,1,NaN


In [37]:
data['hot_weather'] = data['hmax'].apply(hot_weather)
data['heavyrain_weather'] = data['weather'].apply(heavyrain_weather)
data['typhoon_weather'] = data['time_end_day'].apply(typhoon_weather)
data.head()

,ticket_record_id,begin_station_id,end_station_id,unit_price,tk_num,userid,original_type,pay_type,target,description,...,hmin,weather,wind_direction,wind_strength,hmean,rain,good,hot_weather,heavyrain_weather,typhoon_weather
0,0000086a-025f-11e7-bc41-005056900b11,408.0,314.0,500,1,NaN,4,NaN,TVM,NaN,...,14.0,小雨,东北风,2级,16.0,1,NaN,0,0,0
1,00001488-2d89-11e7-bc41-005056900b11,825.0,410.0,300,1,e7e9f4d8-e6a9-4b65-9e7b-b7a088652239,4,NaN,TVM,NaN,...,21.0,多云,东南风,3级,25.0,0,1.0,0,0,0
2,000015dd-f1c6-11e6-ab56-005056900b11,314.0,622.0,200,1,NaN,4,NaN,TVM,NaN,...,9.0,晴,北风,1级,15.0,0,1.0,0,0,0
3,000017f4-1f57-11e7-bc41-005056900b11,105.0,245.0,400,1,40d9f40e-65a5-42f6-8b75-46782c3143f2,3,NaN,TVM,NaN,...,17.0,中雨,北风,2级,18.5,1,NaN,0,0,0
4,00002d20-0403-11e7-bc41-005056900b11,302.0,3004.0,600,1,NaN,4,NaN,TVM,NaN,...,13.0,小雨,北风,1级,14.5,1,NaN,0,0,0


In [38]:
with pd.ExcelWriter('./数据（1-6月）/weather/goodOrBad_people.xlsx') as writer:
    temp1 = ((data['time_end_hour'][data['good'] == 1.0].value_counts()) /
             (data['time_end_day'][data['good'] == 1.0].duplicated()
              ).value_counts()[False]).sort_index()
    temp1 = temp1.apply(int)
    temp1.name = u'好天气'
    temp2 = ((data['time_end_hour'][data['good'] == 0.0].value_counts()) /
             (data['time_end_day'][data['good'] == 0.0].duplicated()
              ).value_counts()[False]).sort_index()
    temp2.name = u'坏天气'
    temp2 = temp2.apply(int)
    temp = pd.concat([temp1, temp2], axis=1)
    temp.to_excel(writer, sheet_name=str('天气好坏'))

In [39]:
with pd.ExcelWriter('./数据（1-6月）/weather/test.xlsx') as writer:
    temp = data.loc[:, ['time_end_day', 'weather', 'hmax', 'tk_num'
                        ]][(data.time_end_day >= datetime.date(2017, 5, 1))
                           & (data.time_end_day <= datetime.date(2017, 6, 30))]
    temp = temp.groupby(['time_end_day', 'weather', 'hmax']).sum()
    temp.to_excel(writer, sheet_name=str('5、6月'))

In [40]:
# data = data[(data.time_end_day >= datetime.date(2017, 5, 1)) & (data.time_end_day <= datetime.date(2017, 6, 30))]

In [41]:
with pd.ExcelWriter('./数据（1-6月）/weather/weather_people.xlsx') as writer:
    temp1 = ((data['time_end_hour'][data['good'] == 1.0].value_counts()) /
             (data['time_end_day'][data['good'] == 1.0].duplicated()
              ).value_counts()[False]).sort_index()
    temp1.name = u'正常'
    temp1 = temp1.apply(int)

    temp2 = ((
        data['time_end_hour'][data['heavyrain_weather'] == 1].value_counts()) /
             (data['time_end_day'][data['heavyrain_weather'] == 1]
              .duplicated()).value_counts()[False]).sort_index()
    temp2.name = u'暴雨'
    temp2 = temp2.apply(int)

    temp3 = ((
        data['time_end_hour'][data['typhoon_weather'] == 1].value_counts()) /
             (data['time_end_day'][data['typhoon_weather'] == 1].duplicated()
              ).value_counts()[False]).sort_index()
    temp3.name = u'台风'
    temp3 = temp3.apply(int)

    temp4 = ((data['time_end_hour'][data['hot_weather'] == 1].value_counts()) /
             (data['time_end_day'][data['hot_weather'] == 1].duplicated()
              ).value_counts()[False]).sort_index()
    temp4.name = u'高温'
    temp4 = temp4.apply(int)

    temp = pd.concat([temp1, temp2, temp3, temp4], axis=1)
    temp.to_excel(writer, sheet_name=str('天气'))